In [ ]:
n = 1
large_encoder = False

In [ ]:
import os
import json 
import glob
from pathlib import Path

In [ ]:
def get_sents(file):
    with open(file, 'r') as f:
        for line in f:
            doc = json.loads(line)
            sents = list()
            if isinstance(doc['knowledge_graph']['title'][0]['value'], str):
                sents.append(doc['knowledge_graph']['title'][0]['value'])
                sents.extend(doc['split_sentences'])
                for sent in sents:
                    sent = str(sent).replace('\n', ' ').replace('\t', ' ')
                    while '  ' in sent:
                        sent = sent.replace('  ', ' ')
                    yield sent

In [ ]:
l = 'large' if large_encoder else 'small'
project_dir = Path(f'TestProject00{n}{l}/')
os.makedirs(project_dir, exist_ok=True)

jl_files = ['Data/sample_news_2019-03-04.jl', 'Data/sample_news_2019-03-05.jl', 'Data/sample_news_2019-03-06.jl']
jl_files

In [ ]:
for jl in jl_files:
    fname = Path(jl).stem
    dump_tsv = project_dir/f'{fname}.tsv'
    with open(dump_tsv, 'x') as dt:
        sent_gen = get_sents(jl)
        for i, sent in enumerate(sent_gen):
            dt.write(f'{i}\t{sent}\n')

In [ ]:
tsv_files = glob.glob(f'{project_dir}/*.tsv')
tsv_files

In [ ]:
from SimSent.indexer.index_builder import IndexBuilder
from SimSent.vectorizer.sentence_vectorizer import SentenceVectorizer

In [ ]:
sv = SentenceVectorizer(large=large_encoder)

In [ ]:
ibdr = IndexBuilder(project_dir, sentence_vectorizer=sv)

In [ ]:
for tsv in tsv_files:
    ibdr.tsv_to_index(tsv)

In [ ]:
from SimSent.server.service_handler import QueryHandler

from SimSent.vectorizer.sentence_vectorizer import DockerVectorizer
from SimSent.indexer.deploy_handler import RangeShards

In [ ]:
# Ensure correct docker is running the vectorizer container in background 
dv = DockerVectorizer(large=large_encoder)
rs = RangeShards(project_dir, nprobe=8, get_nested=True)

In [ ]:
qp = QueryHandler(dv, rs, project_dir=project_dir, get_nested=True)

In [ ]:
keys = list(rs.shards.keys())
keys

In [ ]:
query = 'Tesla stocks are rising faster every day my dude'

In [ ]:
results = qp.query_corpus(query, keys)

In [ ]:
for Key, Vals in results.items():
    print(f'\n{Key}:')
    for v in Vals:
        print(f'   * ID:    {v[1]}\n'
              f'   * Score: {v[0]}\n'
              f'   * Text:  {v[2]}\n')